## Module loading

In [ ]:
import joblib
import matplotlib
import matplotlib.pyplot as plt
import h5py
import numpy as np
from natsort import natsorted

## Data loading

The data must be in the following format :
- one **train.hdf5** file containing the training data :
    - *"predictors_order"*, a list with the orders of the predictors (description of each patch feature, length m)
    - *"x"*, an array with all the extracted patches, flattened (dimension nxm)
    - *"y"*, an array with all the extracted endpoint voxels (dimension n)
    - *"index_in_givenlist"*, unique patient identifier for each voxel (length n)
- one **test.hdf5** file containing the test data :
    - *"x"*, an array with all the extracted patches (dimension NxXxYxZxM)
    - *"y"*, an array with all the extracted endpoint voxels (dimension NxXxYxZ)
    - *"mask"*, a mask of the brain for each subject with 0 for background, 1 for normal brain, 2 for hypoperfused area (dimension NxXxYxZ)
- one **metadata.dat** file containing an unique variable :
    - *"train_patientnames"* containing patient names (corresponding to indices)

In [ ]:
sourcedir = "data/" # Data directory
model_path = "models/" # output directory

with h5py.File(sourcedir+"train.h5", "r") as traindata:
    pred_list = list(traindata['predictors_order'])
    X, y = np.array(traindata['x']), np.array(traindata['y'])
    indexinlist = np.array(traindata['index_in_givenlist'])
with h5py.File(sourcedir+"test.h5", "r") as testdata:
    testX, testy = np.array(testdata['x']), np.array(testdata['y'])
    testmask = np.array(testdata['mask'])
with h5py.File(srcdir+"metadata.h5", "r") as namesdata:
    total_names = list(namesdata['train_patientnames'])
    total_indices = np.arange(len(total_names))

## Create and train models

In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
xgb = XGBClassifier(
                    learning_rate = 0.1,
                    n_estimators=1000,
                    max_depth=5,
                    n_jobs=-1,#jobs,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective= 'binary:logistic',
                    scale_pos_weight=1,
                    random_state=0
                   )
xgbresults = xgb.fit(X, y, verbose=2)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=50)
rfcresults = rfc.fit(X, y)

## Scoring

In [ ]:
from sklearn.metrics import jaccard_score, roc_auc_score, accuracy_score
def scores(X, y, mask, resultsvar):
    Xflat = X.reshape((-1, X.shape[-1]))
    yflat = y.flatten()
    maskflat = mask.flatten()
    yhat = resultsvar.predict_proba(Xflat)[:,1]
    
    try:
        roc = roc_auc_score(yflat, yhat)
    except ValueError:
        roc = accuracy_score(yflat, np.rint(yhat))
        
    aucS = -1
    aucR = -1
    mask = paramarray["mask"]
    if np.sum(yflat[np.logical_and(maskflat>0,maskflat<2)]>0.5) != 0:
        aucS = roc_auc_score(yflat[np.logical_and(maskflat>0,maskflat<2)]>0.5, yhat[np.logical_and(maskflat>0,maskflat<2)])
    if np.sum(yflat[maskflat==2]>0.5) != 0:
        aucR = roc_auc_score(yflat[maskflat==2]>0.5, yhat[maskflat==2])
    if aucS == -1 and aucR == -1:
        auc0 = 0
    elif aucS == -1:
        auc0 = aucR
    elif aucR == -1:
        auc0 = aucS
    else:
        auc0 = (aucS+aucR)/2
        
    return {"jaccard":jaccard_score(yflat, yhat>0.5), 
            "auroc":roc,
            "auc0":auc0,
            "volumegold":np.sum(yflat),
            "volumepred":np.sum(yhat>0.5)}

In [ ]:
for i in range(testX.shape[0]):
    print("Test subject", i)
    print("Gradient boosting:")
    print(scores(testX[i], testy[i], testmask[i], xgbresults))
    print("Random forest:")
    print(scores(testX[i], testy[i], testmask[i], rfcresults))

## Model export

In [ ]:
joblib.dump(xgb, compress=3, protocol=2, filename=model_path+"/XGBmodel.dump.gz")
joblib.dump(rfc, compress=3, protocol=2, filename=model_path+"/RFmodel.dump.gz")